1. Q learning against strategies: Cooperator(),Defector(),TitForTat(),Grudger(),Random().
2. Experiment with hyperparameters:agent = QLearner(alpha=0.1, gamma=0.9, epsilon=1.0) Linear epsilon decay agent.epsilon = 1.0    agent.epsilon_decay = 0.995
3. use of function approximation ReLU 
4. Q table printed with more reps and turns, no deep learning
5. adding a replay buffer as storage
6. Use a separate target network to stablize learning feedforward neural network is created with 2 hidden layers,2-element          history vector
7. imporve training Evaluating every 10 episodes during training Tracking average rewards from evaluation episodes Plotting        rewards to see training progress. Different parameters.
8. calculating mean, t-statistic,p - value and plotting the graph for 5 iterations
9. calculating mean, t-statistic,p - value and plotting the graph for 50 iterations
10. Q-learner compared to other strategies: Alternator, Anti Tit-for-tat, Bully, CooperatorHunter, CyclerCCCD
11. Q-learner compared to other strategies: Handshake, Inverse, Golden, Pi, WinStayLoseShift
12. Q-learner compared to other strategies MetaHunter, StochasticCooperator, NMWEDeterministic, Desperate, Hopeless
13. Training multiple agents and then playing tournament against each other also swapping repetitions and turns of each agent.
14. Q learner against all the oppponents together
15. Q learner against all the oppponents together but change in noise level and change in parameters

1. Q learning against strategies: Cooperator(),Defector(),TitForTat(),Grudger(),Random()

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot

# Define the Q-learning agent
class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]

    # Create the tournament
    tournament = axl.Tournament(
        players=[agent] + opponents,
        repetitions=10,
        turns=200,
        match_attributes={"game": axl.Game(), "noise": 0.02},
    )

    # Run the tournament
    results = tournament.play()
    print(results.scores)
    print(np.array(results.scores).sum(axis=1))
    
    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()
# Run the Q-learning tournament
q_learning_tournament()

2. Experiment with hyperparameters:
agent = QLearner(alpha=0.1, gamma=0.9, epsilon=1.0)
Linear epsilon decay 
agent.epsilon = 1.0
agent.epsilon_decay = 0.995

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

# Define the Q-learning agent
class QLearner(axl.Player):
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=1.0):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()
    #Linear epsilon decay 
    agent.epsilon = 1.0
    agent.epsilon_decay = 0.995

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]
    
    mean_scores = []

    for t in range(10):
        agent.reset_metrics()

        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))
        
        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()
# Run the Q-learning tournament
q_learning_tournament()

3. use of function approximation ReLU 

In [ ]:
import tensorflow as tf
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

# Define the Q-learning agent
class NeuralQLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.5):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score
        self.model = tf.keras.Sequential([
          tf.keras.layers.Dense(24, input_dim=2, activation='relu'),  
          tf.keras.layers.Dense(1)
        ])

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score    

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return NeuralQLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    
 

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = NeuralQLearner()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]

    mean_scores = []

    for t in range(10):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)
    
    # Print the results
    print(results.ranked_names)

    # Plot the results
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

4. Q table printed with more reps and turns, no deep learning

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

# Define the Q-learning agent
class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.5):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score
        
    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
 

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]

    mean_scores = []

    for t in range(5):
        agent.reset_metrics()
    # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=100,
            turns=50,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))
        
        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    print(agent.q_table)
    # Plot the results
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

5. adding a replay buffer as storage

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
from collections import deque
import scipy.stats as stats

# Define the Q-learning agent
class DQNAgent(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score
        self.memory = deque(maxlen=2000)

        
    def remember(self, state, action, reward, next_state):
        self.memory.append((state, action, reward, next_state))
    
    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]
        if len(self.memory) > 2000:  
             batch = random.sample(self.memory, batch_size)  
        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)
        
        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return DQNAgent(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = DQNAgent()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
    ]

    mean_scores = []

    for t in range(10):
        agent.reset_metrics()
    # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        agent.memory
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))
    
        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()
# Run the Q-learning tournament
q_learning_tournament()

6. Use a separate target network to stablize learning feedforward neural network is created with 2 hidden layers,2-element history vector

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import tensorflow as tf
from tensorflow import keras
import scipy.stats as stats

# Define the Q-learning agent
class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score
        self.model = keras.Sequential([
            keras.layers.Dense(24, input_shape=(len(self.history),), activation='relu'), 
            keras.layers.Dense(24, activation='relu'),
            keras.layers.Dense(2)
            ])
        
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.01), 
            loss='mse')

        self.target_model = keras.models.clone_model(self.model)

    def strategy(self, opponent):
        state = tuple(np.array(self.history).reshape(1, -1)[0])
        if state not in self.q_table:
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            action = np.random.choice([0, 1]) 
        else:
            q_values = self.model.predict(np.array([state]))
            action = np.argmax(q_values[0])

        if self.last_state is not None:
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        self.last_state = state
        self.last_action = action
        
        if self.last_state is not None:
            last_q_value = self.q_table[self.last_state][self.last_action]  
            reward_delta = self.total_score - last_q_value

            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta
        
        with tf.GradientTape() as tape:
            target_q_values = self.target_model(np.array([self.last_state]))
            current_q_value = target_q_values[0][self.last_action]
            loss = (reward_delta + self.gamma * np.max(target_q_values[0]) - current_q_value) ** 2
    
        gradients = tape.gradient(loss, self.model.variables)
        #gradients = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        optimizer.apply_gradients(zip(grads, self.model.variables))

        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
    ]
    
    mean_scores = []

    for t in range(2):
        agent.reset_metrics()

    # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)
    
    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()
# Run the Q-learning tournament
q_learning_tournament()

7. imporve training Evaluating every 10 episodes during training
Tracking average rewards from evaluation episodes
Plotting rewards to see training progress. Different parameters.

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

# Define the Q-learning agent
class QLearner(axl.Player):
    def __init__(self, alpha, gamma, epsilon):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
        
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    
    for i in range(100):     
           if i % 10 == 0:
                for alpha in [0.1, 0.2, 0.3]:
                       for gamma in [0.1, 0.2, 0.3]:
                            for epsilon in [0.1, 0.2, 0.3]:
                                  agent = QLearner(alpha, gamma, epsilon)

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]

    mean_scores = []

    for t in range(10):
        agent.reset_metrics()
    # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)    

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()
# Run the Q-learning tournament
q_learning_tournament()

8. calculating mean, t-statistic,p - value and plotting the graph for 5 iterations

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(5):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        results = tournament.play()# Run the tournament
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        agent.store_metrics(score, outcome)# Store metrics for each iteration

        mean_score = np.mean(score)# Calculate mean score
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

9. calculating mean, t-statistic,p - value and plotting the graph for 50 iterations

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(50):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

10. Q-learner compared to other strategies: Alternator, Anti Tit-for-tat, Bully, CooperatorHunter, CyclerCCCD

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Alternator(),
        axl.AntiTitForTat(),
        axl.Bully(),
        axl.CooperatorHunter(),
        axl.CyclerCCCD(),
        ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(5):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

11. Q-learner compared to other strategies: Handshake, Inverse, Golden, Pi, WinStayLoseShift

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.Handshake(),
        axl.Inverse(),
        axl.Golden(),
        axl.Pi(),
        axl.WinStayLoseShift(),
        ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(5):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

12. Q-learner compared to other strategies MetaHunter, StochasticCooperator, NMWEDeterministic, Desperate, Hopeless

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()

    # Create the opponent strategies
    opponents = [
        axl.FoolMeOnce(),
        axl.OnceBitten(),
        axl.Willing(),
        axl.Hopeless(),
        axl.Desperate(),
        ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(5):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=10,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )

        # Run the tournament
        results = tournament.play()
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        # Store metrics for each iteration
        agent.store_metrics(score, outcome)

        # Calculate mean score
        mean_score = np.mean(score)
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

13. Training multiple agents and then playing tournament against each other also swapping repetitions and turns of each agent.

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    
    agent1 = QLearner()
    agent2 = QLearner()
    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
    ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(5):
        agent1.reset_metrics()
        # Create the tournament
        tournament1 = axl.Tournament(
            players=[agent1] + opponents,
            repetitions=200,
            turns=100,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )
        
    for t in range(5):
        agent2.reset_metrics()
        # Create the tournament
        tournament2 = axl.Tournament(
            players=[agent2] + opponents,
            repetitions=100,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.05},
        )
    
        tournament = axl.Tournament(players=[agent1, agent2], repetitions=100, turns=100)

        results = tournament.play()# Run the tournament
        print(results.scores)
        print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        agent1.store_metrics(score, outcome)# Store metrics for each iteration
        agent2.store_metrics(score, outcome)
        
        mean_score = np.mean(score)# Calculate mean score
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent1.ttest_scores(scores1, scores2)
        agent2.ttest_scores(scores1, scores2)
        
    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

14. Q learner against all the oppponents together

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()
    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
        axl.FoolMeOnce(),
        axl.OnceBitten(),
        axl.Willing(),
        axl.Hopeless(),
        axl.Desperate(),
        axl.Alternator(),
        axl.AntiTitForTat(),
        axl.Bully(),
        axl.CooperatorHunter(),
        axl.CyclerCCCD(),
        axl.Handshake(),
        axl.Inverse(),
        axl.Golden(),
        axl.Pi(),
        axl.WinStayLoseShift(),
    ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(10):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=50,
            turns=100,
            match_attributes={"game": axl.Game(), "noise": 0.02},
        )
        if t % 10 == 0:
            agent.q_table = {} 

        results = tournament.play()# Run the tournament
        #print(results.scores)
        #print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        agent.store_metrics(score, outcome)# Store metrics for each iteration

        mean_score = np.mean(score)# Calculate mean score
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()

15. Q learner against all the oppponents together but change in noise level and change in parameters

In [ ]:
import axelrod as axl
import numpy as np
import matplotlib.pyplot as plot
import scipy.stats as stats

class QLearner(axl.Player):
    def __init__(self, alpha=0.25, gamma=0.45, epsilon=0.995):
        super().__init__()
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}  # Q-table to store state-action values
        self.last_state = None
        self.last_action = None
        self.total_score = 0  # Track total score

    def strategy(self, opponent):
        state = tuple(self.history)
        if state not in self.q_table:
            # Initialize the Q-values for this state
            self.q_table[state] = [0, 0]

        if np.random.uniform() < self.epsilon:
            # Explore by choosing a random action
            action = np.random.choice([0, 1])
        else:
            # Exploit by choosing the action with the highest Q-value
            q_values = self.q_table[state]
            action = np.argmax(q_values)

        if self.last_state is not None:
            # Update the Q-value of the last state-action pair
            last_q_value = self.q_table[self.last_state][self.last_action]
            reward_delta = self.total_score - last_q_value
            self.q_table[self.last_state][self.last_action] += self.alpha * reward_delta

        # Remember the current state-action pair
        self.last_state = state
        self.last_action = action

        # Return the chosen action 
        return action

    def set_match_attributes(self, **kwargs):
        if "game" in kwargs:
            self.game = kwargs["game"]
            del kwargs["game"]
        super().set_match_attributes(**kwargs)

    def reset(self):
        super().reset()
        self.total_score = 0  # Reset total score

    def receive_match_attributes(self):
        self.total_score = 0  # Reset total score
        super().receive_match_attributes()

    def receive_result(self, opponent, actions, results):
        super().receive_result(opponent, actions, results)
        self.total_score += results[self]

    def clone(self):
        return QLearner(self.alpha, self.gamma, self.epsilon)
    
    def reset_metrics(self):
        self.match_scores = []
        self.match_outcomes = []

    def store_metrics(self, score, outcome):
        self.match_scores.append(score)
        self.match_outcomes.append(outcome)

    def ttest_scores(self, sample1, sample2):
        tstat, pval = stats.ttest_ind(sample1, sample2)
        print("t-statistic:", tstat)
        print("p-value:", pval) 

        
def plot_learning_curve(mean_scores):
        plot.plot(mean_scores)
        plot.title("Learning Curve")
        plot.xlabel("Tournament")
        plot.ylabel("Mean Score")
        plot.show()
    

# Define the Q-learning tournament
def q_learning_tournament():
    # Create the Q-learning agent
    agent = QLearner()
    # Create the opponent strategies
    opponents = [
        axl.Cooperator(),
        axl.Defector(),
        axl.TitForTat(),
        axl.Grudger(),
        axl.Random(),
        axl.FoolMeOnce(),
        axl.OnceBitten(),
        axl.Willing(),
        axl.Hopeless(),
        axl.Desperate(),
        axl.Alternator(),
        axl.AntiTitForTat(),
        axl.Bully(),
        axl.CooperatorHunter(),
        axl.CyclerCCCD(),
        axl.Handshake(),
        axl.Inverse(),
        axl.Golden(),
        axl.Pi(),
        axl.WinStayLoseShift(),
    ]
    
    # Create an empty list to store mean scores across iterations
    mean_scores = []

    for t in range(10):
        agent.reset_metrics()
        # Create the tournament
        tournament = axl.Tournament(
            players=[agent] + opponents,
            repetitions=100,
            turns=200,
            match_attributes={"game": axl.Game(), "noise": 0.05},
        )

        results = tournament.play()# Run the tournament
        #print(results.scores)
        #print(np.array(results.scores).sum(axis=1))

        score = np.array(results.scores)
        outcome = max(results.scores)

        agent.store_metrics(score, outcome)# Store metrics for each iteration

        mean_score = np.mean(score)# Calculate mean score
        mean_scores.append(mean_score)

        # Statistical analysis
        scores1 = score[:5].flatten()
        scores2 = score[5:].flatten()
        agent.ttest_scores(scores1, scores2)

    # Plot learning curve after all iterations
    plot_learning_curve(mean_scores)

    # Print the results
    print(results.ranked_names)
    plot = axl.Plot(results)
    p = plot.boxplot()
    p.show()

# Run the Q-learning tournament
q_learning_tournament()